# Attempt 1 - fipy software
After spending roughly 20 hours trying to set up and simulate the 1D stready state Saint-Venant Equations on fipy, I talked to Elieen and we decided it would be best to scrap fipy and use pretested software for these equations since fipy is complex. However, she said to still turn this work in with some additional documentation to show that I attempted to do it from "scratch" and tried my best. 

First, I imported fipy and the functions I planed to use.

In [10]:
# imports
#!pip install fipy
from fipy import Variable, FaceVariable, CellVariable, Grid1D, ExplicitDiffusionTerm, TransientTerm, DiffusionTerm, Viewer, ImplicitSourceTerm
from fipy.tools import numerix

Here I define the constants for the simulation. We have gravitional acceleration, mannings coefficent and the channel length. The mannings coefficient is for a concrete. Then the length of the channel is 1000 meters so we elimate as much wave feedback as possible.  

In [11]:
# constants

# g (m/s^2)
g = 9.81
# mannings coefficient
n = 0.012
# length of the channel (m)
channel_length = 1000  

Here we define the mesh which provides the basis for the grid which we can set boundry conditions to and will help the software solve the PDE's. Then we generate the mesh in the cell below. 

In [12]:
# mesh set up
nx = 100
Lx = channel_length

In [13]:
# create the mesh
mesh = Grid1D(nx=nx, Lx=Lx)

Here we create the varibles. In these PDE's the varibles are the water depth and the flow velocity. Initally, we had set values for both h and u but realized since it is a steady state, there would be no "inital values". 

In [14]:
# creaye variables
h = CellVariable(mesh=mesh, name='water depth') #had value = 0 and one but removed becuase its steady state
u = CellVariable(mesh=mesh, name='flow velocity')

Then we define the friction slope or Sf. Sf is found from Mannings equation using the constants defined above.

In [15]:
# define/create Sf
Sf = (n**2 * abs(u) * u) / (h**(4/3))

Here is where we had the most difficulty due to the number of options avaible in fipy. Initally we had both equations only using diffusion terms since the documentaion primarly using diffusion terms. I believe that is because they are the simplst to use. However, in the eq1, the term should be the advection term since it is dealing with the transport of mass. In the momentum equation or eq2, the first term should also be an advection term and the last should be a source. I am not sure what the middle term should be. The advection terms have many pages of examples and very few, written descriptions so I was quite lost. This is when Eileen said I should probably use a different software. Here is a link to the all of the documentation of advection terms: https://www.ctcms.nist.gov/fipy/fipy/generated/fipy.terms.html#fipy.terms.firstOrderAdvectionTerm.FirstOrderAdvectionTerm 

In [16]:
# create pde's (eq2 may be wrong)
eq1 = DiffusionTerm(coeff=1.0, var=u) == 0
eq2 = DiffusionTerm(coeff=1.0, var=u**2 * h + (g * h**2) / 2) + g * h * Sf == 0

Next I defined the boundry conditions based on other examples they provided for 2 PDE's couppled.

In [17]:
# set boundary conditions (boundry conditions)

# height at x=channel_legth?
h.faceGrad.constrain(1, where=mesh.facesRight)
# hight at start?
h.constrain(100, where=mesh.facesLeft)
# steam velocity at x=channel_length?
u.faceGrad.constrain(1, where=mesh.facesRight)
# stream velocity at start?
u.faceGrad.constrain(100, where=mesh.facesLeft)

This step is where I was getting an error, but after some research, I belive it was because of my origional Eq1 and Eq2 being incorrect and them never converging bsed on this article. https://stackoverflow.com/questions/63187771/runtimeerror-factor-is-exactly-singular-and-overflow-encountered-in-square-in

In [9]:
# Solve the equations
eq = eq1 & eq2
eq.solve()

C:\Users\15093\anaconda3\Lib\site-packages\fipy\variables\variable.py:1143: RuntimeWarning: invalid value encountered in divide
  return self._BinaryOperatorVariable(lambda a, b: a / b, other)


RuntimeError: Factor is exactly singular

Finally, this step wpuld show the water depth and stream velocity in steady state across the length of the specified channel. However, since I could not get the coupled equations to work, they do not change what so ever.

In [ ]:
# view stream depth and view stream water velocity (not sure on min and max data values)
viewer_h = Viewer(vars=h, datamin=0, datamax=2)
viewer_u = Viewer(vars=u, datamin=-1.5, datamax=1.5)
viewer_h.plot()
viewer_u.plot()